In [5]:
import pandas as pd
import numpy as np
from citipy import citipy
import time
from datetime import datetime
import requests

from config import weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [7]:
cities = []

for coordinate in lat_lngs:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    if city not in cities:
        cities.append(city)

len(cities)

727

In [8]:
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
       
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | butaritari
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | bonavista
Processing Record 4 of Set 1 | kapaa
Processing Record 5 of Set 1 | provideniya
Processing Record 6 of Set 1 | lompoc
Processing Record 7 of Set 1 | tuktoyaktuk
Processing Record 8 of Set 1 | nerchinskiy zavod
Processing Record 9 of Set 1 | yellowknife
Processing Record 10 of Set 1 | indramayu
Processing Record 11 of Set 1 | port alfred
Processing Record 12 of Set 1 | constitucion
Processing Record 13 of Set 1 | tsihombe
City not found. Skipping...
Processing Record 14 of Set 1 | camacha
Processing Record 15 of Set 1 | sitka
Processing Record 16 of Set 1 | punta arenas
Processing Record 17 of Set 1 | kavieng
Processing Record 18 of Set 1 | avarua
Processing Record 19 of Set 1 | bambous virieux
Processing Record 20 of Set 1 | corn island
Processing Record 21 of Set 1 | murray bridge
Processing Record 22 of S

In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Butaritari,KI,3.0707,172.7902,81.34,81,83,20.31,light rain
1,Rikitea,PF,-23.1203,-134.9692,77.31,70,20,6.82,few clouds
2,Bonavista,CA,48.6499,-53.1147,33.80,72,68,25.93,broken clouds
3,Kapaa,US,22.0752,-159.3190,77.47,65,0,12.66,clear sky
4,Provideniya,RU,64.3833,-173.3000,23.36,86,100,13.00,heavy snow
5,Lompoc,US,34.6391,-120.4579,66.56,60,20,18.41,few clouds
6,Tuktoyaktuk,CA,69.4541,-133.0374,-14.80,70,0,17.27,clear sky
7,Nerchinskiy Zavod,RU,51.3088,119.6100,-27.38,100,59,1.68,broken clouds
8,Yellowknife,CA,62.4560,-114.3525,-11.58,75,20,11.50,few clouds
9,Indramayu,ID,-6.3264,108.3200,78.66,83,86,7.54,overcast clouds


In [11]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")